# Topic Modelling Neighborhood Overview: NERs

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns; sns.set()
import glob as glob

In [2]:
def load_sample_data(country, city):
    directory = 'data/' + country + '/' + city + '/'
    csv_files = [file_name for file_name in glob.glob(directory + '*') if file_name[-4:] == '.csv']
    
    for file_loc in csv_files:
        file_name = file_loc.split('/')[3]
        
        if file_name == 'listings_sample.csv':
            #Read Listing Sample
            listing_sample = pd.read_csv(file_loc)
            
        elif file_name == 'reviews_sample.csv':
            #Read Review Sample
            reviews_sample = pd.read_csv(file_loc)
            
        elif file_name == 'neighbourhoods_sample.csv':
            #Read Neighborhoods
            neighbourhoods_sample = pd.read_csv(file_loc)
            
    return [listing_sample, reviews_sample, neighbourhoods_sample]


def load_full_data(country, city):
    directory = 'data/' + country + '/' + city + '/'
    csv_files = [file_name for file_name in glob.glob(directory + '*') if file_name[-4:] == '.csv']
    
    for file_loc in csv_files:
        file_name = file_loc.split('/')[3]
        
        if file_name == 'listings_full.csv':
            #Read Listings
            listings_full = pd.read_csv(file_loc).drop(columns=['Unnamed: 0'])
            
        elif file_name == 'reviews_full.csv':
            #Read Reviews
            reviews_full = pd.read_csv(file_loc).drop(columns=['Unnamed: 0'])
            
        elif file_name == 'calendar_full.csv':
            #Read Calendar
            calendar_full = pd.read_csv(file_loc).drop(columns=['Unnamed: 0'])
            
    return [listings_full, reviews_full, calendar_full]

## Load Data

In [5]:
# 
listings_sf = pd.read_csv('../sf/listings_detailed.csv')

In [6]:
def clean_listing_full(listing_full_data):
    """Cleans listing_full.csv data"""
    # Input Data
    df = listing_full_data
    
    # String to Datetime
    df['last_scraped'] = pd.to_datetime(df['last_scraped'])
    df['host_since'] = pd.to_datetime(df['host_since'])
    df['calendar_last_scraped'] = pd.to_datetime(df['calendar_last_scraped'])
    df['first_review'] = pd.to_datetime(df['first_review'])
    df['last_review'] = pd.to_datetime(df['last_review'])
    
    # String to Numeric
    df['host_response_rate'] = pd.to_numeric(df['host_response_rate'].str[:-1]) / 100
    df['price'] = pd.to_numeric(df['price'].str[1:].str.replace(',',''))
    df['weekly_price'] = pd.to_numeric(df['weekly_price'].str[1:].str.replace(',',''))
    df['monthly_price'] = pd.to_numeric(df['monthly_price'].str[1:].str.replace(',',''))
    df['security_deposit'] = pd.to_numeric(df['security_deposit'].str[1:].str.replace(',',''))
    df['cleaning_fee'] = pd.to_numeric(df['cleaning_fee'].str[1:].str.replace(',',''))
    df['extra_people'] = pd.to_numeric(df['extra_people'].str[1:].str.replace(',',''))

    # t/f to Numeric
    df['host_is_superhost'] = (df['host_is_superhost'] == "t").astype(int)
    df['host_has_profile_pic'] = (df['host_has_profile_pic'] == "t").astype(int)
    df['host_identity_verified'] = (df['host_identity_verified'] == "t").astype(int)
    df['is_location_exact'] = (df['is_location_exact'] == "t").astype(int)
    df['has_availability'] = (df['has_availability'] == "t").astype(int)
    df['requires_license'] = (df['requires_license'] == "t").astype(int)
    df['instant_bookable'] = (df['instant_bookable'] == "t").astype(int)
    df['is_business_travel_ready'] = (df['is_business_travel_ready'] == "t").astype(int)
    df['require_guest_profile_picture'] = (df['require_guest_profile_picture'] == "t").astype(int)
    df['require_guest_phone_verification'] = (df['require_guest_phone_verification'] == "t").astype(int)
    
    return df

In [7]:
# Clean Listing SF
clean_listings_sf = clean_listing_full(listings_sf)

In [8]:
listings_detailed = clean_listings_sf.copy()

ID = list(listings_detailed.iloc[:,:2].columns)

ABOUT_COLS = list(listings_detailed.iloc[:,3:15].columns)

PICS_COLS = list(listings_detailed.iloc[:,15:19].columns)

HOST_COLS = list(listings_detailed.columns[listings_detailed.columns.str.contains('host')])

NEIGHBORHOOD_COLS = list(listings_detailed.columns[listings_detailed.columns.str.contains('neighbourhood')])

LOCATION_COLS = list(listings_detailed.iloc[:,37:51].columns)

PROPERTY_COLS = list(listings_detailed.iloc[:,51:60].columns)

PRICE_COLS = list(listings_detailed.iloc[:,60:67].columns)

NIGHTS_COLS = list(listings_detailed.columns[listings_detailed.columns.str.contains('mum')])

AVAILABILITY_COLS = list(listings_detailed.columns[listings_detailed.columns.str.contains('availability')])

REVIEW_COLS = list(listings_detailed.columns[listings_detailed.columns.str.contains('review')])

SCRAPING_COLS = ['scrape_id','calendar_updated','calendar_last_scraped']

ELSE_COLS = ['requires_license', 'license', 'jurisdiction_names', 'instant_bookable',\
             'is_business_travel_ready', 'cancellation_policy', 'require_guest_profile_picture',\
             'require_guest_phone_verification']

## Neighborhood Info

In [9]:
neighborhood_info = clean_listings_sf[ID + NEIGHBORHOOD_COLS + ABOUT_COLS]

In [10]:
neighborhood_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7198 entries, 0 to 7197
Data columns (total 18 columns):
id                              7198 non-null int64
listing_url                     7198 non-null object
host_neighbourhood              6559 non-null object
neighbourhood                   6660 non-null object
neighbourhood_cleansed          7198 non-null object
neighbourhood_group_cleansed    0 non-null float64
last_scraped                    7198 non-null datetime64[ns]
name                            7198 non-null object
summary                         7000 non-null object
space                           6109 non-null object
description                     7183 non-null object
experiences_offered             7198 non-null object
neighborhood_overview           5310 non-null object
notes                           4486 non-null object
transit                         5238 non-null object
access                          4794 non-null object
interaction                     4931 non

In [11]:
# 
neighborhood_overviews = neighborhood_info[['id','neighbourhood_cleansed','neighborhood_overview']].dropna()

## Clean & Tokenize Overview Text

In [12]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
import re

# Initialize Lemmatizer
lemmatizer = WordNetLemmatizer()

# Get Stopwords
stop_words = set(stopwords.words('english'))

# Get Puncuations
punctuations = set(string.punctuation)

def clean_text(doc):
    
    # remove all ascii
    doc = re.sub(r'[^\x00-\x7F]+',' ', doc)

    # Tokenize, Lemmatize, and Remove Stopwords
    tokens = [lemmatizer.lemmatize(word).lower() for word in nltk.word_tokenize(doc) if word.lower() not in set(stop_words | punctuations)]
    
    return tokens

In [13]:
# Clean & Tokenize Overviews
neighborhood_overviews['tokens'] = neighborhood_overviews['neighborhood_overview'].apply(clean_text)

In [14]:
neighborhood_overviews['clean_overviews'] = neighborhood_overviews['tokens'].map(lambda x: ' '.join(x))

## Get POS

In [34]:
def get_pos_sentences(data):
    # Tokenize each sentence into words: token_sentences
    token_sentences = [nltk.word_tokenize(re.sub(r'[^\x00-\x7F]+',' ', sent)) for sent in data]

    # Tag each tokenized sentence into parts of speech: pos_sentences
    pos_sentences = [nltk.pos_tag(sent) for sent in token_sentences]
    return pos_sentences

In [71]:
# Initialize TextExtraction with DS Skills Listing
pos_overviews_neighborhood = get_pos_sentences(neighborhood_overviews['neighborhood_overview'])

### NER

In [72]:
def get_name_entities(pos_sentences):
    """Return Name Entities List"""
    # Create the named entity chunks: chunked_sentences
    chunked_sentences = nltk.ne_chunk_sents(pos_sentences, binary=True)
        
    # Chunk List per Listing
    listing_chunk_list = []

    # Test for stems of the tree with 'NE' tags
    for sent in chunked_sentences:
        chunks_list = []
        for chunk in sent:
            if hasattr(chunk, "label") and chunk.label() == "NE":
                chunks_list.append(chunk.leaves())

        #Append List of Chunks
        listing_chunk_list.append(chunks_list)
        
    # Chunks to List of Words
    listing_name_entity_list = [[' '.join([word[0] for word in chunk]) for chunk in chunk_list] for chunk_list in listing_chunk_list]
                
    return listing_name_entity_list

In [73]:
ner_overviews_neighborhood = get_name_entities(pos_overviews_neighborhood)

In [74]:
neighborhood_overviews = neighborhood_overviews.reset_index(drop=True)

In [75]:
neighborhood_overviews['name_entities'] = pd.Series(ner_overviews_neighborhood)

# Topic Modeling

In [76]:
token_texts = list(neighborhood_overviews['name_entities'].values)

In [77]:
from gensim.corpora.dictionary import Dictionary

# Create a corpus from a list of texts
common_dictionary = Dictionary(token_texts)
common_corpus = [common_dictionary.doc2bow(text) for text in token_texts]

In [78]:
from gensim.models.ldamulticore import LdaMulticore
import time
ldam = LdaMulticore

num_topics = 50
num_words = 10
passes = 50

# Get Start Time
start_time = time.time()

# LDA Model
ldam_model = ldam(common_corpus, num_topics=num_topics, id2word=common_dictionary, passes=passes)
model_end_time = time.time() # Model End Time

# LDA Results
results = ldam_model.print_topics(num_topics=num_topics, num_words=num_words)
result_time = time.time() # Results Time

In [79]:
ldam_model.save('../models/ldam_neighborhood_overviews_50topics_10words_50passes_ner.model')

In [80]:
def display_results(results):
    for index, results in results:
        print(str(index) + ': ' + str(', '.join(results.split('"')[1::2])))

In [81]:
display_results(results)

0: Cole Valley, Golden Gate Park, Haight Ashbury, Haight Street, San Francisco, Zazie, Downtown SF, Cole, Whole Foods, Ocean Beach
1: Farmer, Cole Street, United States, Please, San Francisco, Mission District, Bay Area, Bernal Heights Hill, Downtown, Glen Park Station
2: West Portal, Muni, Lake Merced, Castro Street, SF Zoo, Dolores Park, San Francisco, Stonestown Galleria, Trick Dog, Davidson
3: Safeway, Walgreens, Opera House, City Hall, Davies Symphony Hall, Hotel, Asian Art Museum, Trader Joe, Beautiful, Muni
4: Mission District, San Francisco, Dolores Park, Valencia, Dolores, BART, Mission Dolores, Downtown, Valencia Corridor, SF
5: Tenderloin, Brenda, Ferry Building, West Field Shopping Center, Bourbon, Branch, French Soul Food, Borobudur, Indonesian, Noe
6: Bernal Heights, Bernal Hill, San Francisco, Bernal, Cortland Avenue, Mission District, Cortland, Bernal Heights Park, Mission, Cortland Ave
7: URL, Philz, Yerba Buena, SoMa, PHONE, San Francisco, Giants, Hip, SF MOMA, Russia

In [88]:
def display_results_no_duplicates(results):
    all_lists = []
    for index, result in results:
        all_lists = all_lists + result.split('"')[1::2]
    
    # Get Counts of each word
    counts = pd.Series(all_lists).value_counts()
    no_duplicates = counts[counts == 1].index
    
    for index, result in results:
        print(str(index) + ': ' + str(', '.join([word for word in result.split('"')[1::2] if word in no_duplicates])))

In [89]:
display_results_no_duplicates(results)

0: Cole Valley, Zazie, Downtown SF, Cole
1: Farmer, Cole Street, United States, Please, Bernal Heights Hill, Glen Park Station
2: Lake Merced, Castro Street, SF Zoo, Stonestown Galleria, Trick Dog, Davidson
3: Walgreens, Opera House, City Hall, Davies Symphony Hall, Hotel, Asian Art Museum, Beautiful
4: Valencia, Dolores, Mission Dolores, Valencia Corridor
5: Tenderloin, Brenda, West Field Shopping Center, Bourbon, Branch, French Soul Food, Borobudur, Indonesian, Noe
6: Bernal Heights, Bernal Hill, Bernal, Cortland Avenue, Cortland, Bernal Heights Park, Cortland Ave
7: URL, Philz, Yerba Buena, PHONE, Giants, Hip, SF MOMA, Russian Hill
8: SOMA, Moscone Center, Moscone
9: Polk Street, Irish, LGBT, US, Folsom, Alemany, BiRite, GG Park
10: Airbnb, Fillmore, Japantown, Website, Lower Pacific Heights, Fillmore Street, Fillmore District
11: Park, Cable, Van Ness, Grace Cathedral, Swan Oyster Depot, Tommy
12: Alta Plaza, Ashbury, Filmore Street, Hayes Street, Design District
13: Transamerica P

# By Neighborhood

In [82]:
tokens_by_neighborhood = neighborhood_overviews.groupby('neighbourhood_cleansed')[['name_entities']].apply(sum)

# Topic Modeling

In [83]:
token_texts_by_neighborhood = list(tokens_by_neighborhood['name_entities'].values)

In [84]:
from gensim.corpora.dictionary import Dictionary

# Create a corpus from a list of texts
common_dictionary_by_neighborhood = Dictionary(token_texts_by_neighborhood)
common_corpus_by_neighborhood = [common_dictionary_by_neighborhood.doc2bow(text) for text in token_texts_by_neighborhood]

In [85]:
from gensim.models.ldamulticore import LdaMulticore
import time
ldam = LdaMulticore

num_topics = 50
num_words = 10
passes = 50

# Get Start Time
start_time = time.time()

# LDA Model
ldam_model_by_neighborhood = ldam(common_corpus_by_neighborhood, num_topics=num_topics, id2word=common_dictionary_by_neighborhood, passes=passes)
model_end_time = time.time() # Model End Time

# LDA Results
results_by_neighborhood = ldam_model_by_neighborhood.print_topics(num_topics=num_topics, num_words=num_words)
result_time = time.time() # Results Time

In [86]:
ldam_model_by_neighborhood.save('../models/ldam_overview_by_neighborhood_50topics_10words_50passes_ner.model')

In [87]:
display_results(results_by_neighborhood)

0: North Beach, San Francisco, Fisherman, Coit Tower, Chinatown, Lombard Street, Telegraph Hill, Russian, Italian, City Lights Bookstore
1: San Francisco, Hayes Valley, Dolores Park, Mission District, Alamo Square, Mission, NOPA, Golden Gate Park, Castro, Haight
2: San Francisco, Castro, Dolores Park, Noe Valley, Golden Gate Park, Mission District, Bernal Heights, Mission, Whole Foods, Haight
3: San Francisco, Golden Gate Bridge, Marina, Pacific Heights, Fisherman, Chestnut Street, Marina Green, Fort Mason, Cow Hollow, Ghirardelli Square
4: San Francisco, Hayes Valley, Victorian, NOPA, Golden Gate Park, Fillmore, Alamo Square, Haight, Divisadero, Castro
5: San Francisco, Mission Street, Twin Peaks, Cow Palace, Golden Gate Bridge, Fisherman, Hewett, Convenient, Balboa Park, SFO
6: San Francisco, Mission Bay, Westfield San Francisco Centre, Market, South, Moscone Conference Center, SoMa, Civic Center Metro Station, SoMA, Bayview
7: San Francisco, Golden Gate Park, Ocean Beach, Hayes Vall

In [90]:
display_results_no_duplicates(results_by_neighborhood)

0: Lombard Street, Telegraph Hill, City Lights Bookstore
1: 
2: 
3: Marina, Chestnut Street, Marina Green, Fort Mason, Ghirardelli Square
4: Divisadero
5: Cow Palace, Hewett, Convenient, Balboa Park
6: 
7: 
8: Silver Terrace, Bayview District, Monk, Website
9: 
10: 
11: Lake Merced, Target, Ocean, San Francisco State University, Korean, Stern Grove
12: Cliff House
13: Love
14: 
15: 
16: 
17: 
18: 
19: 
20: Portola, Little Hollywood, Tiffany, Caltrain Station, Visitacion Valley, Muni
21: Glen Canyon Park
22: Valencia Street, California
23: 
24: Duboce Triangle, Corona Heights, SF, Duboce Park, Castro District
25: 
26: 
27: Laurel Heights, Laurel Village, California Street, Sacramento Street, Presidio Heights, Kaiser Hospital, Clement
28: SOMA, Dolores
29: 
30: Cortland Avenue, Cortland, Bernal Heights Park
31: 
32: New York, Bay Area Rapid Transit, Visit Fisherman
33: MUNI, Diamond Heights Shopping Center, City
34: Balboa
35: 
36: 
37: Brenda, Bourbon
38: 
39: 
40: French
41: Great
42: 